In [ ]:
# Python
from flask import Flask, Response
from time import sleep
import io
import picamera

# Initialize Flask app
app = Flask(__name__)

# Function to generate video frames
def generate_frames():
  camera = picamera.PiCamera()
  camera.resolution = (320, 240)  # Change resolution as needed
  camera.framerate = 10  # Change framerate as needed

  # Start capturing frames
  stream = io.BytesIO()
  for foo in camera.capture_continuous(stream, format='jpeg', use_video_port=True):
    # Return the current frame
    yield (b'--frame\r\n'
           b'Content-Type: image/jpeg\r\n\r\n' + foo.raw + b'\r\n')
    # Reset stream for the next frame
    stream.seek(0)
    stream.truncate()

    # Sleep to maintain desired framerate
    sleep(1/camera.framerate)

  camera.close()

@app.route('/video_feed')
def video_feed():
  # Video streaming route
  return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
  # Start Flask app
  app.run(host='0.0.0.0', debug=False)